In [8]:
import SimpleITK as sitk
import os


#Try to use absolute paths as much as possible

#img: string with filepath to main image
#mask: string with filepath to segmentation
#labeldir: string with filepath to labels
#tempdir: string with filepath that you don't mind having a random file be written to, defaults to just C: - good idea to change it
    #labeldir and tempdir must be in the main C: drive - workarounds were made to use img and mask with G: drive for convenience
def display(img, mask,labeldir=None,tempdir="C:\\",name="temp viewer"):
    v = sitk.ImageViewer()
    r = sitk.ImageFileReader()
    r.SetFileName(img)
    image = r.Execute()

    r.SetFileName(mask)
    w = sitk.ImageFileWriter()
    
    os.chdir(tempdir)
    w.SetFileName("temp.mha")
    w.Execute(r.Execute())

    v.SetApplication(r'C:\Program Files\ITK-SNAP 3.8\bin\ITK-SNAP')
    s = ""
    if labeldir != None:
        s = ' -l '+ labeldir
    v.SetCommand('itk-snap -s '+ tempdir + '\\temp.mha' + s + " -g") #this requires the ITK-SNAP directory to be added to the PATH env var.
    v.SetTitle(name)
    v.Execute(image)
    os.remove("temp.mha")
    


In [10]:
files = ["001","002","003","004","006","008","011","012","013","015","022","023","080","105","127","149","152","169","173"]

for f in files:
        display(r"C:\Users\theev\PersonalBackup\Case Western Reserve University\INVentLab - Documents\MichaelKong\coronalv2.5\Images\UH_{}_0000.mha".format(f),
                r"C:\Users\theev\PersonalBackup\Case Western Reserve University\INVentLab - Documents\MichaelKong\coronalv2.5\UH_{}.mha".format(f),
                labeldir=r"C:\Users\theev\Downloads\Rectal_Data_Labels.label",tempdir=r"C:\Users\theev\OneDrive\Documents\LAB\temp")
        print()
        input()

In [2]:
import numpy as np
from PIL import Image
import SimpleITK as sitk
from PIL import ImageFilter, ImageOps

def hasColor(img)->bool:
    img = ImageOps.grayscale(img)
    img = np.array(img)
    return img.max() > 0

def visualizeAsGifBoundary(mainimg:str,savedir:str,segimg:str = None,name:str="array.gif",gndtrth:str=None, speed=150, crop:bool=True):
    r = sitk.ImageFileReader()
    r.SetFileName(mainimg)
    main = r.Execute()
    main = sitk.GetArrayFromImage(main)

    main = main/main.max()
    main  = main *255
    
    imgs = np.uint8(main)
    imgs = [Image.fromarray(img, 'L').convert("RGBA") for img in imgs]

    red = (255,0,0,255) #missed i.e., FN

    blue = (0,0,255,255) #wrong ID i.e., FP

    green = (0,255,0,255) #Overlap
    purple = (255,0,255,255)
    clear = (0,0,0,0)
    
    first = 10000000000
    last = 0

    if not segimg == None:
        
        r.SetFileName(segimg)
        seg = r.Execute()
        seg = sitk.GetArrayFromImage(seg)
        seg = seg *255
        segs = np.uint8(seg)
        segs = [Image.fromarray(img, 'L').filter(ImageFilter.FIND_EDGES) for img in segs]
        segs = [seg.convert("RGBA") for seg in segs]
        datas = [seg.getdata() for seg in segs]

        
        count = 0
        for data in datas:
            newData=[]
            for item in data:
                
                if item[0] == 0 and item[1] == 0 and item[2] == 0:
                    newData.append(clear)
                else:
                    newData.append(green)
            segs[count].putdata(newData)
            count += 1

        if not gndtrth==None:
            datas = [seg.getdata() for seg in segs]

            r.SetFileName(gndtrth)
            gs = r.Execute()
            gs = sitk.GetArrayFromImage(gs)
            gs = gs *255
            gs = np.uint8(gs)
            gs = [Image.fromarray(g, 'L').filter(ImageFilter.FIND_EDGES) for g in gs]
            gs = [g.convert("RGBA") for g in gs]
            gdatas = [g.getdata() for g in gs]    

            for j in range(0,len(datas)):
                data = datas[j]
                gdata = gdatas[j]
                
                newData=[]
                for i in range(0,len(data)):
                    item = data[i]
                    gitem = gdata[i]
                    if item[0] == 0 and item[1] == 0 and item[2] == 0 and gitem[0] == 0 and gitem[1] == 0 and gitem[2] == 0:
                        newData.append(clear)
                    else:
                        if item[1] != 0 and gitem[1]!=0:
                            newData.append(purple)
                        else:
                            if item[1] !=0:
                                newData.append(blue)
                            else:
                                newData.append(red)
                
                segs[j].putdata(newData)
                
        
        for i in range(0,len(imgs)):
            imgs[i].paste(segs[i],(0,0),segs[i])
            if hasColor(segs[i]):
                first = i if i < first else first
                last = i if i > last else last

    first = first - 5 if (first-5 > 0) else 0
    last = last + 5 if (last+5 < len(imgs)) else len(imgs)
        
    if crop:
        imgs[first].save(savedir+"/BOUNDARY"+name, save_all=True, append_images=imgs[first+1:last], duration=speed, loop=0)
    else:
    # duration is the number of milliseconds between frames; this is 40 frames per second
        imgs[0].save(savedir+"/BOUNDARY"+name, save_all=True, append_images=imgs[1:], duration=speed, loop=0)


In [3]:
import glob
mainPath = r"C:\Users\theev\PersonalBackup\Case Western Reserve University\INVentLab - Documents\MichaelKong\coronalv2.5"
ground = glob.glob(mainPath+"\\GroundTruths\\*.mha")
imgs = glob.glob(mainPath+"\\Images\\*.mha")
outs = glob.glob(mainPath+"\\*.mha")
ground.sort()
imgs.sort()
outs.sort()
files = ["001","002","003","004","006","008","011","012","013","015","022","023","080","105","127","149","152","169","173"]
for i in range(0,len(ground)):
    visualizeAsGifBoundary(imgs[i], r"C:\Users\theev\PersonalBackup\Case Western Reserve University\INVentLab - Documents\MichaelKong\test_visualizations",
            segimg=outs[i],gndtrth=ground[i],name="UH_"+files[i]+".gif", speed=150)

In [9]:
#testing slicer


import glob
import SimpleITK as sitk
startdir = r"C:\Users\theev\PersonalBackup\Case Western Reserve University\INVentLab - Documents\MichaelKong\coronalv2.5"
enddir = r"C:\Users\theev\PersonalBackup\Case Western Reserve University\INVentLab - Documents\MichaelKong\3DSlicer"

imgs = glob.glob(startdir+"\\Images\\*.mha")
outs = glob.glob(startdir+"\\*.mha")
for f in imgs:
    t = f.replace(startdir+"\\Images\\","").replace(".mha","")
    r = sitk.ReadImage(f)
    sitk.WriteImage(r,enddir+"\\"+t+".nii.gz")

for f in outs:
    t = f.replace(startdir+"\\","").replace(".mha","")
    r = sitk.ReadImage(f)
    sitk.WriteImage(r,enddir+"\\"+t+"_0000_mask.nii.gz")
    
